# 🚗 Nhận Dạng Biển Số Xe Việt Nam

## Vietnamese License Plate Recognition

### 🎯 Tính năng:
- Phát hiện biển số xe trong ảnh
- Nhận dạng ký tự và số trên biển số
- Hỗ trợ biển số 1 dòng và 2 dòng

### 📋 Hướng dẫn:
1. Chọn GPU Runtime: Runtime → Change runtime type → GPU
2. Chạy từng cell theo thứ tự
3. Đợi cell hoàn thành trước khi chạy cell tiếp theo

---

## 1. Setup và Clone Repository

In [ ]:
import os
import sys

# Clone repository
if not os.path.exists('BsxGoogleColab'):
    print("🔄 Cloning repository...")
    !git clone https://github.com/ngoclam99/BsxGoogleColab.git
    print("✅ Clone successful!")
else:
    print("✅ Repository already exists!")

# Change directory
os.chdir('BsxGoogleColab')
print(f"📁 Current directory: {os.getcwd()}")

# Add to Python path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

## 2. Install Dependencies

In [ ]:
# Install required packages
print("📦 Installing dependencies...")
!pip install -q opencv-python-headless matplotlib numpy Pillow torch torchvision tqdm pyyaml requests pandas seaborn
print("✅ Installation complete!")

## 3. Import Libraries

In [ ]:
# Import libraries
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# Import helper functions
try:
    import function.utils_rotate as utils_rotate
    import function.helper as helper
    print("✅ Helper functions imported successfully!")
except ImportError as e:
    print(f"❌ Error importing helpers: {e}")

# Check files
required_files = ['model/LP_detector.pt', 'model/LP_ocr.pt', 'yolov5']
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file} exists")
    else:
        print(f"❌ {file} missing")

print("✅ Setup complete!")

## 4. Load Models

In [ ]:
# Load YOLOv5 models
print("🤖 Loading models...")
print("⏳ This may take 2-3 minutes...")

try:
    # Load detection model
    yolo_LP_detect = torch.hub.load('yolov5', 'custom', path='model/LP_detector.pt', force_reload=True, source='local')
    print("✅ LP Detection model loaded!")
    
    # Load OCR model
    yolo_license_plate = torch.hub.load('yolov5', 'custom', path='model/LP_ocr.pt', force_reload=True, source='local')
    print("✅ LP OCR model loaded!")
    
    # Set confidence
    yolo_license_plate.conf = 0.60
    
    print("🎉 All models ready!")
    
except Exception as e:
    print(f"❌ Error loading models: {e}")
    print("💡 Try: Runtime → Restart Runtime and run again")

## 5. Detection Function

In [ ]:
def detect_license_plate(image_path):
    """Detect and recognize license plate"""
    
    print(f"🔍 Processing: {image_path}")
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Cannot read image: {image_path}")
        return None, None
    
    try:
        # Detect license plates
        plates = yolo_LP_detect(img, size=640)
        list_plates = plates.pandas().xyxy[0].values.tolist()
        list_read_plates = set()
        
        print(f"🎯 Found {len(list_plates)} license plate regions")
        
        if len(list_plates) == 0:
            # Try reading entire image
            lp = helper.read_plate(yolo_license_plate, img)
            if lp != "unknown":
                list_read_plates.add(lp)
        else:
            # Process each detected plate
            for i, plate in enumerate(list_plates):
                x = int(plate[0])
                y = int(plate[1])
                w = int(plate[2] - plate[0])
                h = int(plate[3] - plate[1])
                
                # Crop license plate
                crop_img = img[y:y+h, x:x+w]
                
                # Draw rectangle
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
                
                # Try different image processing
                found = False
                for cc in range(0, 2):
                    for ct in range(0, 2):
                        processed_img = utils_rotate.deskew(crop_img, cc, ct)
                        lp = helper.read_plate(yolo_license_plate, processed_img)
                        if lp != "unknown":
                            list_read_plates.add(lp)
                            found = True
                            break
                    if found:
                        break
        
        # Add text to image
        if list_read_plates:
            y_offset = 30
            for plate_text in list_read_plates:
                cv2.putText(img, plate_text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                y_offset += 40
        
        if list_read_plates:
            print(f"✅ Result: {list_read_plates}")
        else:
            print("❌ No license plate recognized")
        
        return img, list_read_plates
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None, None

print("✅ Detection function ready!")

## 6. Test with Sample Images

In [ ]:
# Find test images
test_images = glob.glob("test_image/*.jpg") + glob.glob("test_image/*.png")

if test_images:
    print(f"🖼️ Found {len(test_images)} test images:")
    for i, img in enumerate(test_images):
        print(f"   {i}: {img}")
    
    # Test first image
    test_img = test_images[0]  # Change index to test different image
    
    print(f"\n🚀 Testing: {test_img}")
    print("=" * 50)
    
    result_img, detected_plates = detect_license_plate(test_img)
    
    if result_img is not None:
        # Display results
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Original image
        original = cv2.imread(test_img)
        axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original Image")
        axes[0].axis('off')
        
        # Result image
        axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
        result_title = f"Result: {detected_plates if detected_plates else 'No detection'}"
        axes[1].set_title(result_title)
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n📊 SUMMARY:")
        print(f"   File: {test_img}")
        print(f"   Size: {original.shape[1]}x{original.shape[0]}")
        print(f"   Plates found: {len(detected_plates) if detected_plates else 0}")
        if detected_plates:
            for i, plate in enumerate(detected_plates, 1):
                print(f"   Plate {i}: {plate}")
else:
    print("❌ No test images found")

## 7. Upload Your Own Image

In [ ]:
from google.colab import files

def upload_and_test():
    """Upload and test your own image"""
    print("📤 Select image file from your computer...")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f"\n🔍 Processing: {filename}")
        print("=" * 40)
        
        # Save uploaded file
        with open(filename, 'wb') as f:
            f.write(uploaded[filename])
        
        # Process image
        result_img, detected_plates = detect_license_plate(filename)
        
        if result_img is not None:
            # Display results
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Original
            original = cv2.imread(filename)
            axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
            axes[0].set_title("Your Image")
            axes[0].axis('off')
            
            # Result
            axes[1].imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
            result_title = f"Result: {detected_plates if detected_plates else 'No detection'}"
            axes[1].set_title(result_title)
            axes[1].axis('off')
            
            plt.tight_layout()
            plt.show()
            
            print(f"\n📊 RESULT:")
            print(f"   File: {filename}")
            if detected_plates:
                for i, plate in enumerate(detected_plates, 1):
                    print(f"   License Plate {i}: {plate}")
            else:
                print("   No license plate detected")

# Uncomment the line below to upload and test your image
# upload_and_test()

## 📝 Instructions

### How to use:
1. **Run cells 1-4** to setup and load models
2. **Run cell 5** to define detection function
3. **Run cell 6** to test with sample images
4. **Run cell 7** to upload your own image (uncomment the last line)

### Tips:
- Choose **GPU Runtime** for faster processing
- Use **clear, high-quality images** for better results
- **License plates should be visible** and not obscured
- **Straight angle photos** work best

### Troubleshooting:
- **Models won't load**: Restart Runtime and try again
- **Import errors**: Check repository structure
- **No detection**: Try different images or better quality photos

---
**🚗 Good luck with license plate recognition! 🎉**